In [ ]:
# kutuphaneler
from glob import glob
from os import link
import requests
from parsel import Selector
import pandas as pd
from fake_useragent import UserAgent
from sympy import re

ua = UserAgent()
header = {"User-Agent": str(ua.chrome)}
# ilan ozelliklerinin kaydedildigi degisken
ilanlar = []

# linklerin kaydedildigi degisken
linkler = []

# 1 sayfadan baslayarak x sayfa kadar tarar, sitede en son 1400 sayfa vardır
# yeni sonuc gelmeyince dongu durdurulur
for i in range(1, 1400):
    resp = requests.get(f"https://www.hepsiemlak.com/istanbul-satilik?page={i}", headers=header)
    selectorLink = Selector(resp.text)
    # x sayfadaki linkler alinir
    tempLink = selectorLink.xpath('//div[@class="links"]/a/@href').getall()
    # yeni sonuc gelmediyse dongu kirilir
    if len(tempLink) < 1:
        pass
    # sonuc varsa linklere eklenir
    linkler += tempLink
    print(f"{i}.sayfa bitti")

print(len(linkler))
# ilanlar birer birer dongu ile gezilir
def tara(linkler):
    
    hataliLinkler = []
    global ilanlar
    if input("Devam et? -1") is "1":
        print("Devam ediliyor...")
    else:
        return
    counter = 1
    for linkIndex in range(len(linkler)):
        ua = UserAgent()
        header = {"User-Agent": str(ua.chrome)}
        print(f"{counter}/{len(linkler)}---{len(ilanlar)}")
        counter+=1
        try:
            resp = requests.get(f"https://www.hepsiemlak.com{linkler[linkIndex]}",headers=header)
            selector = Selector(resp.text)
            # fiyat bilgileri alinir ve gereksiz bosluklar strip ile temizlenir
            fiyat = selector.xpath('//p[@class="fontRB fz24 price"]/text()').get().strip()
            #  il ilce mahalle bilgileri alinir ve gereksiz bosluklar temizlenir
            kisaDetay = [i.replace(" ","").replace("\n","").replace("m","") for i in selector.xpath('//ul[@class="short-info-list"]/li/text()').getall()]
            il = kisaDetay[0]
            ilce = kisaDetay[1]
            mahalle = kisaDetay[2]
            # detayli ilan bilgileri alinir
            detaylar = [i.strip().replace("\n","") for i in selector.xpath('//ul[@class="adv-info-list"]/li/span/text()').getall()]
            # veriler alinirken net ve brut m2 ayri ayri gorunuyor bu veriler bulunur ve listede birlestirilir (cekilen veri ile ilgili)
            for i in detaylar:
                if i[0] == "/":
                    birlestir = detaylar.index(i)
                    break
            detaylar[birlestir-1] += detaylar[birlestir]
            detaylar.pop(birlestir)
            # detaylar sutun ve icerik bilgisi olarak ayrilir sirayla tek cift durumuna gore
            sutunlar = [detaylar[i] for i in range(len(detaylar)) if i%2==0]
            icerik = [detaylar[i] for i in range(len(detaylar)) if i%2==1]

            # ilan bilgileri tempilan degiskenine atilir ve eksik veri olmadigi durumda genel ilanlar listesine atilir
            tempilan = [fiyat, il, ilce, mahalle]
            # sutun yani ozelliklerde dongu ile gezinilir ve istenen bilgiler ozelliklere eklenir
            for i in range(len(sutunlar)):
                if sutunlar[i] == "İlan Durumu":
                    tempilan.append(icerik[i])

                if sutunlar[i] == "Konut Şekli":
                    tempilan.append(icerik[i])
                    
                if sutunlar[i] == "Oda + Salon Sayısı":
                    tempilan.append(icerik[i])
                
                if sutunlar[i] == "Brüt / Net M2":
                    a,b = icerik[i].split("/ ")
                    tempilan.append(a)
                    tempilan.append(b)
                
                if sutunlar[i] == "Bulunduğu Kat":
                    tempilan.append(icerik[i])
                
                if sutunlar[i] == "Bina Yaşı":
                    tempilan.append(icerik[i])
                
                if sutunlar[i] == "Banyo Sayısı":
                    tempilan.append(icerik[i])
                
                if sutunlar[i] == "Yapı Tipi":
                    tempilan.append(icerik[i])
                
                if sutunlar[i] == "Eşya Durumu":
                    tempilan.append(icerik[i])
                
                if sutunlar[i] == "Site İçerisinde":
                    tempilan.append(icerik[i])
                
                if sutunlar[i] == "Yakıt Tipi":
                    tempilan.append(icerik[i])
            # ozelliklerin butunlugu kontrol edilir 16 ozelligin hepsi varsa genel listeye eklenir
            if len(tempilan) == 16:
                ilanlar.append(tempilan)
                sonuc = pd.DataFrame(data= ilanlar)
                #print(sonuc)
                # ilgili dosyalara donusturulur cikti alinir
                sonuc.to_csv("outputCSV.csv")
                sonuc.to_excel("outputExcel.xlsx")
        except Exception as e:
            print("HATA: ", e)
            hataliLinkler.append(linkler[linkIndex])
    return tara(linkler=hataliLinkler)

tara(linkler=linkler)
# dataframe olusturulur
sonuc = pd.DataFrame(data= ilanlar)
print(sonuc)
# ilgili dosyalara donusturulur cikti alinir
sonuc.to_csv("outputCSV.csv")
sonuc.to_excel("outputExcel.xlsx")


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("outputCSV.csv")
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [4]:
# Sütun Adlarını Düzenleme
df_2 = df.rename(columns = {"0":"fiyat",
                     "1":"il",
                     "2":"ilce",
                     "3":"mahalle",
                     "4":"ilan_durumu",
                     "5":"konut",
                     "6":"oda_sayisi",
                     "7":"net_mt",
                     "8":"brut_mt",
                     "9":"dairenin_katı",
                     "10":"yas",
                     "11":"esya_durumu",
                     "12":"banyo_sayisi",
                     "13":"yapi_tipi",
                     "14":"site_icerisinde",
                     "15":"yakit_tipi"})

In [5]:
df_2.head()

,fiyat,il,ilce,mahalle,ilan_durumu,konut,oda_sayisi,net_mt,brut_mt,dairenin_katı,yas,esya_durumu,banyo_sayisi,yapi_tipi,site_icerisinde,yakit_tipi
0,1.350.000 TL,İstanbul,Beylikdüzü,Cuhuriyet,Satılık,Daire,3 + 1,150 m2,140 m2,Ara Kat,1 Yaşında,Eşyalı Değil,1,Betonarme,Evet (Site Adı Belirtilmemiş),Doğalgaz
1,22.000.000 TL,İstanbul,Beşiktaş,Etiler,Satılık,Daire,5 + 1,284 m2,256 m2,4. Kat,14 Yaşında,Eşyalı Değil,3,Betonarme,Badur Boğaziçi Evleri Sitesi,Doğalgaz
2,3.950.000 TL,İstanbul,Maltepe,Feyzullah,Satılık,Daire,3 + 1,165 m2,145 m2,1. Kat,28 Yaşında,Eşyalı Değil,1,Betonarme,Adatepe Sitesi,Doğalgaz
3,50.400.000 TL,İstanbul,Beşiktaş,Levazı,Satılık,Residence,4 + 1,420 m2,350 m2,Bahçe Katı,2 Yaşında,Eşyalı Değil,4,Betonarme,yooistanbul,Doğalgaz
4,4.300.000 TL,İstanbul,Büyükçekece,Ekinoba,Satılık,Daire,3 + 1,160 m2,116 m2,1. Kat,2 Yaşında,Eşyalı,2,Betonarme,Evet (Site Adı Belirtilmemiş),Doğalgaz


In [6]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1201 entries, 0 to 1200
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   fiyat            1201 non-null   object
 1   il               1201 non-null   object
 2   ilce             1201 non-null   object
 3   mahalle          1201 non-null   object
 4   ilan_durumu      1201 non-null   object
 5   konut            1201 non-null   object
 6   oda_sayisi       1201 non-null   object
 7   net_mt           1201 non-null   object
 8   brut_mt          1201 non-null   object
 9   dairenin_katı    1201 non-null   object
 10  yas              1201 non-null   object
 11  esya_durumu      1201 non-null   object
 12  banyo_sayisi     1201 non-null   int64 
 13  yapi_tipi        1201 non-null   object
 14  site_icerisinde  1201 non-null   object
 15  yakit_tipi       1201 non-null   object
dtypes: int64(1), object(15)
memory usage: 150.2+ KB


In [8]:
df_2.describe()

,banyo_sayisi
count,1201.000000
mean,1.773522
std,0.872925
min,1.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,14.000000
